# Animals

Here, we will investigate the emission of GHG that comes from animals. First, we will focus on the **direct** emission (enteric fermentation, manure,...), later we will look at the emission that comes from crops used to feed animals

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# constants
DATA_PATH = '../data/'
RAW_DATA_PATH = DATA_PATH + 'current_FAO/raw_files/'
GENERATED_DATA_PATH = '../generated/'

## Direct GHG

Direct GHG emission includes: Enteric Fermentation, Manure left on pasture and Manure Management. As shown in *greenhouse.ipynb*, the two biggest causes of GHG emission are Enteric Fermentation and Manure left on pasture. 

NOTE: Download file from here for most recent data: http://www.fao.org/faostat/en/#data/GE

In [46]:
enteric_fermentation_path = RAW_DATA_PATH+'Emissions_Agriculture_Enteric_Fermentation_E_ALL_DATA_(Normalized).csv'
enteric_fermentation = pd.read_csv(enteric_fermentation_path, encoding='latin-1', engine='python')

In [47]:
enteric_fermentation.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,1107,Asses,5111,Stocks,1961,1961,Head,1300000.0,NaN
1,2,Afghanistan,1107,Asses,5111,Stocks,1962,1962,Head,851850.0,NaN
2,2,Afghanistan,1107,Asses,5111,Stocks,1963,1963,Head,1001112.0,NaN
3,2,Afghanistan,1107,Asses,5111,Stocks,1964,1964,Head,1150000.0,F
4,2,Afghanistan,1107,Asses,5111,Stocks,1965,1965,Head,1300000.0,NaN


In [48]:
# Check present values
enteric_fermentation.Flag.value_counts()

Fc    279113
A     256601
F      61267
*       1879
Im      1626
Name: Flag, dtype: int64

The flags represent : 
* `A` : Aggregate, may include official, semi-official, estimated or calculated data
* `Fc` : Calculated data
* `F` : FAO estimate
* `Im` : FAO data based on imputation methodology
* `*`: Unofficial figure


In [81]:
print(enteric_fermentation.Unit.value_counts())
#print(enteric_fermentation.Item.value_counts())
#print(enteric_fermentation.Year.value_counts())
enteric_fermentation.Element.value_counts()

gigagrams      364004
Head           167913
kg CH4/head     92249
Name: Unit, dtype: int64


Emissions (CO2eq) (Enteric)                  182002
Emissions (CH4) (Enteric)                    182002
Stocks                                       167913
Implied emission factor for CH4 (Enteric)     92249
Name: Element, dtype: int64

If element is stocks, this means the amount of animals (Unit = Head)

In [52]:
# Drop some unnecessary columns
enteric_fermentation.drop(columns=["Area Code", "Item Code", "Element Code", "Year Code"], inplace=True)

Let us first just look at the year 2017 (most recent data)

In [114]:
enteric_fermentation_2017 = enteric_fermentation[enteric_fermentation.Year == 2017].copy()
enteric_fermentation_2017.drop(columns=["Year"], inplace=True)

In [115]:
# Check for missing data
enteric_fermentation_2017.isna().any()

Area       False
Item       False
Element    False
Unit       False
Value      False
Flag        True
dtype: bool

In [116]:
# Around 2.5% of our flags are unknown, which means we do not know how the data is gatherd
np.count_nonzero(enteric_fermentation_2017.isna().Flag)/len(enteric_fermentation_2017.Flag)

0.026938122332859176

The missing flags are only present when element = stocks

In [117]:
enteric_fermentation_2017[enteric_fermentation_2017.isna().any(axis=1)].Element.value_counts()

Stocks    303
Name: Element, dtype: int64

In [118]:
# Drop the flags in the meantime
enteric_fermentation_2017.drop(columns=['Flag'], inplace=True)

We now want to see the total emission per kind of animal

In [176]:
columns=["Animal", "CO2eq (Gg)", "Stocks"]

In [179]:
# Take all world data
enteric_fermentation_2017_world = enteric_fermentation_2017[enteric_fermentation_2017["Area"]=="World"].copy()
# Drop unecessary colomns (units are known) and pivot 
enteric_fermentation_2017_world_total = enteric_fermentation_2017_world.drop(columns=['Area', 'Unit']).pivot(index="Item", columns="Element")
# Format
enteric_fermentation_2017_world_total.columns = enteric_fermentation_2017_world_total.columns.droplevel()
enteric_fermentation_2017_world_total.reset_index(inplace=True)
# Drop unecessary CH4 column (same as CO2eq multiplied by constant)
enteric_fermentation_2017_world_total.drop(columns=["Emissions (CH4) (Enteric)"], inplace=True)
enteric_fermentation_2017_world_total.columns = columns

In [180]:
enteric_fermentation_2017_world_total

,Animal,CO2eq (Gg),Stocks
0,All Animals,2.100076e+06,NaN
1,Asses,9.617656e+03,4.579836e+07
2,Buffaloes,2.321177e+05,2.009677e+08
3,Camels,3.364576e+04,3.482998e+07
4,Camels and Llamas,3.923297e+04,4.384889e+07
5,Cattle,1.521011e+06,1.491687e+09
6,"Cattle, dairy",3.944052e+05,2.780345e+08
7,"Cattle, non-dairy",1.126606e+06,1.213653e+09
8,Goats,1.086127e+05,1.034407e+09
9,Horses,2.289418e+04,6.056660e+07
